In [1]:
#!pip install Geohash
!pip install python-geohash

In [2]:
import pandas
import geohash as gh

df = pandas.read_table('./master/station_master.tsv',
                                         names=('code', 'name', 'type', 'pref_code', 'pref_name', 'lat', 'lon'))
df['geohash'] = df.apply(lambda x: gh.encode(x.lat, x.lon), axis=1)
#5km精度（5桁）に丸める
df['geohash_5'] = df['geohash'].str[:5]
print(df)

       code     name   type  pref_code pref_name        lat         lon  \
0     26242  相生(兵庫県)  train         28       兵庫県  34.818309  134.474019   
1     24360  相生(岐阜県)  train         21       岐阜県  35.715035  136.950606   
2     21839       相老  train         10       群馬県  36.410476  139.304434   
3     29692      相生山  train         23       愛知県  35.102546  136.976216   
4     26915      秋鹿町  train         32       島根県  35.477213  132.956671   
...     ...      ...    ...        ...       ...        ...         ...   
9179  20617       輪西  train          1       北海道  42.334037  141.007270   
9180  26843       和深  train         30      和歌山県  33.501110  135.654871   
9181  21134       和渕  train          4       宮城県  38.526225  141.220733   
9182  22174        蕨  train         11       埼玉県  35.828153  139.690457   
9183  23759       割出  train         17       石川県  36.602935  136.648340   

           geohash geohash_5  
0     wypwh42zrg0p     wypwh  
1     xn37hmsrhmj6     xn37h  
2     

In [3]:
df.groupby(by='geohash_5').nunique().sort_values('name', ascending=False)

,code,name,type,pref_code,pref_name,lat,lon,geohash
geohash_5,,,,,,,,
xn0m5,44,44,1,1,1,43,43,44
xn77h,41,41,1,1,1,41,41,41
xn0m7,40,40,1,1,1,37,38,40
xpssb,37,37,1,1,1,37,37,37
wyn5w,36,36,1,1,1,36,36,36
...,...,...,...,...,...,...,...,...
xn2f4,1,1,1,1,1,1,1,1
xn2f5,1,1,1,1,1,1,1,1
xn2f8,1,1,1,1,1,1,1,1


In [4]:
df.groupby(by='geohash_5').nunique().reset_index()[['geohash_5', 'code']].sort_values('code', ascending=False) \
    .to_csv('./master/geohash_station_cnt.tsv', sep='\t', header=True, index=False)

In [5]:
#df.to_csv('./master/station_master_with_geohash.tsv', sep='\t', header=True, index=False)

In [6]:
import math
import json
def distance_from_geohash_center(geohash, lat, lon):
    bbox = gh.bbox(geohash)
    center_lat = (bbox['n'] + bbox['s']) / 2.0
    center_lon = (bbox['e'] + bbox['w']) / 2.0
    distance_by_meters = round(math.sqrt(((center_lat - lat) * 111.1) ** 2 + ((center_lon - lon) * 91.1) ** 2) * 1000)
    return distance_by_meters

df['distance'] = df.apply(lambda x: distance_from_geohash_center(x.geohash_5, x.lat, x.lon), axis=1)
df['nearest_from_geohash_center'] = df.groupby(['geohash_5'])['distance'].rank()
df.to_csv('./master/station_master_with_geohash.tsv', sep='\t', header=True, index=False)